#### Travail A

In [43]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn import metrics
import numpy as np
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

> J'ai fait les exercices qu'il est possible de faire python.

#### K-means et Farthest First sur la BD. Weather

J'ai changé rendu la dernière colonne binaire afin de pouvoir calculer le score du clustering.

In [2]:
path_weather = './weather.nominalToBinary.csv'
df_weather = pd.read_csv(path_weather)
df_weather.head()

,outlook=sunny,outlook=overcast,outlook=rainy,temperature=hot,temperature=mild,temperature=cool,humidity=normal,windy=FALSE,play
0,1,0,0,1,0,0,0,1,no
1,1,0,0,1,0,0,0,0,no
2,0,1,0,1,0,0,0,1,yes
3,0,0,1,0,1,0,0,1,yes
4,0,0,1,0,0,1,1,1,yes


In [3]:
df_weather['play'] = df_weather['play'].apply(lambda x: 0 if x == 'no' else 1)
df_weather.head()

,outlook=sunny,outlook=overcast,outlook=rainy,temperature=hot,temperature=mild,temperature=cool,humidity=normal,windy=FALSE,play
0,1,0,0,1,0,0,0,1,0
1,1,0,0,1,0,0,0,0,0
2,0,1,0,1,0,0,0,1,1
3,0,0,1,0,1,0,0,1,1
4,0,0,1,0,0,1,1,1,1


In [4]:
weather_columns = df_weather.columns
print(f'columns = {weather_columns}')

columns = Index(['outlook=sunny', 'outlook=overcast', 'outlook=rainy', 'temperature=hot',
       'temperature=mild', 'temperature=cool', 'humidity=normal',
       'windy=FALSE', 'play'],
      dtype='object')


In [60]:
print(f'Number of samples: {len(df_weather)}')
X, y = df_weather[weather_columns[:-1]], df_weather[weather_columns[-1]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print(f'len(X_train) = {len(X_train)}, len(y_train) = {len(y_train)}')

Number of samples: 14


> Exercice 1 <br>
> L'exercice 9 est inclu dans mon implémentation des exercices 1-2-3 car j'ai assigné une classe à chaque cluster.

#### Farthest-First optimization for Partitioning Around Medoids algorithm
> I implemented it with python standard library, could be faster to implement with numpy.

In [6]:
def dist(a,b):
    """ 
    Distance between vectors a and b (lines of X_train)
    The L1 norm is used, however data points are in {0,1}, so this is the same as the L2 norm.
    """
    # print(f'a = {a}, b = {b}')
    return sum(abs(x-y) for x,y in zip(a,b))

def PAM(list_points, k, seed=0):
    """ 
    Computes the PAM barycenters with farthest-first optimization.
    ******
    PARAMS:
    list_points: list of input feature vectors (= input matrix)
    k: number of barycenters
    """
    N = len(list_points)
    # print(f'N = {N}')
    np.random.seed(seed)
    m1 = np.random.randint(len(list_points))
    print(f'First medoid randomly chosen: {m1}')
    list_medoids = [m1]
    print(f'list_medoids: {list_medoids}')
    distance_matrix = [[dist(list_points[i], list_points[j]) for j in range(N)] for i in range(N)]
    print(f'distance_matrix = {distance_matrix}')
    for i in range(1,k):
        distances = [(sum(distance_matrix[i][m] for m in list_medoids),i) for i in range(N) if i not in list_medoids]
        list_medoids.append(min(distances)[1])
    print(f'\nMedoids after farthest-first: {list_medoids}')

    # Add points to closest medoid
    clusters = [[i] for i in list_medoids]
    for i, p in enumerate(list_points):
        if i not in list_medoids:
            clusters[min((dist(list_points[j], p), k) for k,j in enumerate(list_medoids))[1]].append(i)
    print(f'Clusters after first iteration: {clusters}')
    intra_cluster_variances = [sum(dist(list_points[x],list_points[m]) for x in clusters[i]) for i,m in enumerate(list_medoids)]
    print(f'intra_cluster_variances = {intra_cluster_variances}')
    
    print(f'\n*** SWAP phase ***')
    improvement = True 
    while improvement:
        improvement = False 
        best_swap = 0
        best_var = 0
        swap_indexes = 0, 0
        for i in range(k):
            for j in clusters[i]: 
                # swap p and m: p becomes medoid of i-th cluster, and m becomes normal element of cluster i
                # new medoid of cluster i is list_points[j]
                new_variance_cluster_i = sum(dist(list_points[x], list_points[j]) for x in clusters[i])
                if intra_cluster_variances[i] - new_variance_cluster_i > best_swap:
                    best_swap = intra_cluster_variances[i] - new_variance_cluster_i
                    best_var = new_variance_cluster_i
                    swap_indexes = i,j 
        # make the best swap
        if best_swap > 0:
            print(f'best_swap = {best_swap} > 0, performing swapping with indexes {swap_indexes}')
            print(f'medoids = {list_medoids}')
            list_medoids[swap_indexes[0]] = swap_indexes[1]
            improvement = True
            print(f'new medoids: {list_medoids}')
            # intra_cluster_variances[swap_indexes[0]] = intra_cluster_variances[i] - best_swap
            intra_cluster_variances[swap_indexes[0]] = best_var
            print(f'intra_cluster_variances = {intra_cluster_variances}\n')
    
    return clusters, list_medoids

In [7]:
X_matrix = X_train.values
clusters, list_medoids = PAM(X_matrix, k=2, seed=0)

First medoid randomly chosen: 5
list_medoids: [5]
distance_matrix = [[0, 6, 5, 3, 1, 2, 5, 5, 5, 3, 6], [6, 0, 5, 3, 5, 4, 3, 3, 3, 3, 4], [5, 5, 0, 6, 4, 5, 4, 2, 2, 6, 1], [3, 3, 6, 0, 4, 1, 4, 6, 4, 4, 5], [1, 5, 4, 4, 0, 3, 6, 4, 4, 4, 5], [2, 4, 5, 1, 3, 0, 5, 5, 3, 5, 6], [5, 3, 4, 4, 6, 5, 0, 2, 6, 2, 3], [5, 3, 2, 6, 4, 5, 2, 0, 4, 4, 3], [5, 3, 2, 4, 4, 3, 6, 4, 0, 6, 3], [3, 3, 6, 4, 4, 5, 2, 4, 6, 0, 5], [6, 4, 1, 5, 5, 6, 3, 3, 3, 5, 0]]

Medoids after farthest-first: [5, 3]
Clusters after first iteration: [[5, 0, 2, 4, 7, 8], [3, 1, 6, 9, 10]]
intra_cluster_variances = [18, 16]

*** SWAP phase ***
best_swap = 4 > 0, performing swapping with indexes (1, 6)
medoids = [5, 3]
new medoids: [5, 6]
intra_cluster_variances = [18, 12]

best_swap = 2 > 0, performing swapping with indexes (0, 4)
medoids = [5, 6]
new medoids: [4, 6]
intra_cluster_variances = [16, 12]



#### Affichage des clusters
> Cluster 1

In [8]:
cluster_1 = X_train.iloc[clusters[0]]
cluster_1

,outlook=sunny,outlook=overcast,outlook=rainy,temperature=hot,temperature=mild,temperature=cool,humidity=normal,windy=FALSE
0,1,0,0,1,0,0,0,1
2,0,1,0,1,0,0,0,1
4,0,0,1,0,0,1,1,1
12,0,1,0,1,0,0,1,1
9,0,0,1,0,1,0,1,1
8,1,0,0,0,0,1,1,1


> cluster 2

In [9]:
cluster_2 = X_train.iloc[clusters[1]]
cluster_2

,outlook=sunny,outlook=overcast,outlook=rainy,temperature=hot,temperature=mild,temperature=cool,humidity=normal,windy=FALSE
1,1,0,0,1,0,0,0,0
10,1,0,0,0,1,0,1,0
13,0,0,1,0,1,0,0,0
11,0,1,0,0,1,0,0,0
5,0,0,1,0,0,1,1,0


#### Calcul des centroids 

In [10]:
cluster_1.mean()

outlook=sunny       0.333333
outlook=overcast    0.333333
outlook=rainy       0.333333
temperature=hot     0.500000
temperature=mild    0.166667
temperature=cool    0.333333
humidity=normal     0.666667
windy=FALSE         1.000000
dtype: float64

In [11]:
cluster_2.mean()

outlook=sunny       0.4
outlook=overcast    0.2
outlook=rainy       0.4
temperature=hot     0.2
temperature=mild    0.6
temperature=cool    0.2
humidity=normal     0.4
windy=FALSE         0.0
dtype: float64

#### Calcul de l'erreur
> On prend pour play=yes le cluster qui le plus de bonnes predictions (comme ce que fait Weka). <br>
> Le clustering est non-supervisé donc les valeurs regroupees n'ont pas de label, il faut l'attribuer manuellement.

In [12]:
def get_predictions(clusters):
    predictions = []
    N = len(clusters[0]) + len(clusters[1])
    for i in range(N):
        if i in clusters[0]:
            predictions.append(0)
        else:
            predictions.append(1)
    return predictions

predictions = get_predictions(clusters)

[0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1]

In [13]:
def print_metrics(y_train, y_pred):
    accuracy = sum(y_pred == y_train.to_numpy())/len(y_pred)
    print(f'accuracy = {accuracy:.2f}')
    print(f'confusion matrix: \n{confusion_matrix(y_train, y_pred)}')
    print(classification_report(y_train, y_pred))

In [14]:
print_metrics(y_train, predictions)

accuracy = 0.27
confusion matrix: 
[[1 3]
 [5 2]]
              precision    recall  f1-score   support

           0       0.17      0.25      0.20         4
           1       0.40      0.29      0.33         7

    accuracy                           0.27        11
   macro avg       0.28      0.27      0.27        11
weighted avg       0.32      0.27      0.28        11



#### Calcul de l'accuracy sur test_set
> Les médoids sont utilisés pour déterminer l'appartenance à un cluster

In [15]:
medoids = X_train.iloc[list_medoids].values

array([[0, 1, 0, 1, 0, 0, 1, 1],
       [0, 0, 1, 0, 1, 0, 0, 0]])

In [16]:
X_test

,outlook=sunny,outlook=overcast,outlook=rainy,temperature=hot,temperature=mild,temperature=cool,humidity=normal,windy=FALSE
3,0,0,1,0,1,0,0,1
7,1,0,0,0,1,0,0,1
6,0,1,0,0,0,1,1,0


In [17]:
def predict(medoids, input_vector):
    return min((dist(m, input_vector), i) for i,m in enumerate(medoids))[1]
cluster_predictions_test = np.array([predict(medoids, x) for x in X_test.values])
cluster_predictions_test

array([1, 1, 0])

In [18]:
accuracy_farthest = sum(cluster_predictions_test==y_test)/len(y_test)
print(f'Accuracy of Farthest-First implemetation on test set: {accuracy_farthest}')
print_metrics(y_test, cluster_predictions_test)

Accuracy of Farthest-First implemetation on test set: 0.3333333333333333
accuracy = 0.33
confusion matrix: 
[[0 1]
 [1 1]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.50      0.50      0.50         2

    accuracy                           0.33         3
   macro avg       0.25      0.25      0.25         3
weighted avg       0.33      0.33      0.33         3



> Exercice 2

> La méthode K-means est appliquée avec l'initialisation aléatoire (random) des médoids.

In [19]:
simple_kmeans = KMeans(n_clusters=2, random_state=5, init='random', n_init='auto').fit(X_train)
print(f'kmeans.labels_ = {simple_kmeans.labels_}\ny_train = {y_train.to_list()}')
cluster_1_simple = [i for i in range(len(X_train)) if simple_kmeans.labels_[i] == 0]
cluster_2_simple = [i for i in range(len(X_train)) if simple_kmeans.labels_[i] == 1]
df_1_simple = X_train.iloc[cluster_1_simple]
df_2_simple = X_train.iloc[cluster_2_simple]

kmeans.labels_ = [0 1 1 0 0 0 1 1 1 1 1]
y_train = [1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0]


,outlook=sunny,outlook=overcast,outlook=rainy,temperature=hot,temperature=mild,temperature=cool,humidity=normal,windy=FALSE
10,1,0,0,0,1,0,1,0
4,0,0,1,0,0,1,1,1
13,0,0,1,0,1,0,0,0
9,0,0,1,0,1,0,1,1
8,1,0,0,0,0,1,1,1
11,0,1,0,0,1,0,0,0
5,0,0,1,0,0,1,1,0


> Pandas permets d'afficher les centroids pour chaque valeur des attributs.

In [20]:
df_1_simple.mean()

outlook=sunny       0.50
outlook=overcast    0.50
outlook=rainy       0.00
temperature=hot     1.00
temperature=mild    0.00
temperature=cool    0.00
humidity=normal     0.25
windy=FALSE         0.75
dtype: float64

In [21]:
df_2_simple.mean()

outlook=sunny       0.285714
outlook=overcast    0.142857
outlook=rainy       0.571429
temperature=hot     0.000000
temperature=mild    0.571429
temperature=cool    0.428571
humidity=normal     0.714286
windy=FALSE         0.428571
dtype: float64

In [22]:
kmean_train_accuracy = sum(simple_kmeans.labels_ == y_train)/len(y_train)
print(f'The accuracy of the kmean clustering on the training set is {max(kmean_train_accuracy, 1-kmean_train_accuracy):.3f}')
print_metrics(y_train, simple_kmeans.labels_)

The accuracy of the kmean clustering on the training set is 0.636
accuracy = 0.64
confusion matrix: 
[[2 2]
 [2 5]]
              precision    recall  f1-score   support

           0       0.50      0.50      0.50         4
           1       0.71      0.71      0.71         7

    accuracy                           0.64        11
   macro avg       0.61      0.61      0.61        11
weighted avg       0.64      0.64      0.64        11



Obtention des centroids

On remarque que les centroids kmeans ont des valeurs entre 0 et 1, mais rarement 0.0 et jamais 1.0: les centroids n'appartiennent pas à l'ensemble des instances. <br>
Cela montre que l'algorithme classe individus qui ont des classes différentes pour certains attributs dans le même cluster. Cela justifie l'utilisation de plusieurs classes.

Calcul de l'accuracy sur le test set.

In [23]:
y_pred = simple_kmeans.predict(X_test)
kmean_test_accuracy = sum(y_pred == y_test)/len(y_test)
print(f'y_pred = {y_pred}\ty_test = {y_test.to_list()}')
print(f'L\'accuracy de la méthode kmean sur le test set est: {max(kmean_test_accuracy, 1-kmean_test_accuracy):.3f}')
print(confusion_matrix(y_test, y_pred))

y_pred = [1 1 1]	y_test = [1, 0, 1]
L'accuracy de la méthode kmean sur le test set est: 0.667
[[0 1]
 [0 2]]


L'algorithme fait 1 erreur sur 3 prédictions.

> Exercice III

L'implémentation de Farthest-First conduit à de meilleurs résultats: accuracy de 100% sur le test set, et 72% sur le train set. <br>
On s'attendait à ces meilleurs résultats car Farthest-First est une amélioration de simple KMean. <br>
Comme il y a peu de données (11), le reste des métriques n'est pas très représentatif.

> Exercice 5

In [24]:
simple_kmeans_5 = KMeans(n_clusters=2, random_state=20, init='random', n_init='auto').fit(X_train)
print(f'centroids: {simple_kmeans_5.cluster_centers_}')

cluster_1_simple_2 = [i for i in range(len(X_train)) if simple_kmeans.labels_[i] == 0]
cluster_2_simple_2 = [i for i in range(len(X_train)) if simple_kmeans.labels_[i] == 1]
df_1_simple_2 = X_train.iloc[cluster_1_simple_2]
df_2_simple_2 = X_train.iloc[cluster_2_simple_2]
print(f'centroid 1: \n {df_1_simple_2.mean()}')
print(f'\ncentroid 2: \n {df_2_simple_2.mean()}')

centroids: [[0.40 0.60 -0.00 0.80 0.20 0.00 0.20 0.60]
 [0.33 0.00 0.67 0.00 0.50 0.50 0.83 0.50]]
centroid 1: 
 outlook=sunny       0.50
outlook=overcast    0.50
outlook=rainy       0.00
temperature=hot     1.00
temperature=mild    0.00
temperature=cool    0.00
humidity=normal     0.25
windy=FALSE         0.75
dtype: float64

centroid 2: 
 outlook=sunny       0.285714
outlook=overcast    0.142857
outlook=rainy       0.571429
temperature=hot     0.000000
temperature=mild    0.571429
temperature=cool    0.428571
humidity=normal     0.714286
windy=FALSE         0.428571
dtype: float64


> Exercice X <br>
> Pour cet exercice, compte tenu du faible nombre de données (14), j'utilise un train set uniquement (pas de test set).

In [25]:
seeds = [1, 10, 20, 50, 100, 1000]
accuracies_kmeans = []
accuracies_PAM = []
for seed in seeds:
    kmeans = KMeans(n_clusters=2, random_state=seed, init='random', n_init='auto').fit(X)
    clusters_fs, medoids_fs = PAM(X.values, k=2, seed=seed)

    accuracy_kmean = sum(kmeans.labels_ == y)/len(y)
    predictions_farthest_first = get_predictions(clusters_fs)
    accuracy_fs = sum(predictions_farthest_first == y)/len(y)

    accuracies_PAM.append(max(accuracy_fs, 1-accuracy_fs))
    accuracies_kmeans.append(max(accuracy_kmean, 1-accuracy_kmean))

First medoid randomly chosen: 5
list_medoids: [5]
distance_matrix = [[0, 1, 2, 4, 5, 6, 6, 2, 3, 5, 4, 5, 3, 5], [1, 0, 3, 5, 6, 5, 5, 3, 4, 6, 3, 4, 4, 4], [2, 3, 0, 4, 5, 6, 4, 4, 5, 5, 6, 3, 1, 5], [4, 5, 4, 0, 3, 4, 6, 2, 5, 1, 4, 3, 5, 1], [5, 6, 5, 3, 0, 1, 3, 5, 2, 2, 5, 6, 4, 4], [6, 5, 6, 4, 1, 0, 2, 6, 3, 3, 4, 5, 5, 3], [6, 5, 4, 6, 3, 2, 0, 6, 3, 5, 4, 3, 3, 5], [2, 3, 4, 2, 5, 6, 6, 0, 3, 3, 2, 3, 5, 3], [3, 4, 5, 5, 2, 3, 3, 3, 0, 4, 3, 6, 4, 6], [5, 6, 5, 1, 2, 3, 5, 3, 4, 0, 3, 4, 4, 2], [4, 3, 6, 4, 5, 4, 4, 2, 3, 3, 0, 3, 5, 3], [5, 4, 3, 3, 6, 5, 3, 3, 6, 4, 3, 0, 4, 2], [3, 4, 1, 5, 4, 5, 3, 5, 4, 4, 5, 4, 0, 6], [5, 4, 5, 1, 4, 3, 5, 3, 6, 2, 3, 2, 6, 0]]

Medoids after farthest-first: [5, 4]
Clusters after first iteration: [[5, 1, 6, 10, 11, 13], [4, 0, 2, 3, 7, 8, 9, 12]]
intra_cluster_variances = [19, 26]

*** SWAP phase ***
best_swap = 2 > 0, performing swapping with indexes (0, 10)
medoids = [5, 4]
new medoids: [10, 4]
intra_cluster_variances = [17, 26]

best_

In [26]:
print(f'Accuracies kmean: {accuracies_kmeans}')
print(f'Accuracies_fs: {accuracies_PAM}')
print(f'Maximal accuracy kmean: {max(accuracies_kmeans):.2f}\nMaximal accuracy farthest first: {max(accuracies_PAM):.2f}')

Accuracies kmean: [0.5, 0.6428571428571428, 0.5, 0.5, 0.5714285714285714, 0.6428571428571428]
Accuracies_fs: [0.6428571428571429, 0.7142857142857143, 0.7142857142857143, 0.6428571428571428, 0.5714285714285714, 0.7142857142857143]
Maximal accuracy kmean: 0.64
Maximal accuracy farthest first: 0.71


> Obtention des meilleurs seeds pour les algorithmes aléatoires

In [27]:
def argmax(list):
    return max((x,i) for i,x in enumerate(list))[1]
best_seed_kmean = seeds[argmax(accuracies_kmeans)]
best_seed_fs = seeds[argmax(accuracies_PAM)]
print(f'best_seed_kmean = {best_seed_kmean}\nbest_seed_fs = {best_seed_fs}')

best_seed_kmean = 1000
best_seed_fs = 1000


> Obtention des meilleurs modèles à partir des meilleurs seeds

In [28]:
kmeans = KMeans(n_clusters=2, random_state=best_seed_kmean, init='random', n_init='auto').fit(X)
predictions_kmeans = abs(kmeans.predict(X)-1)
clusters_fs, medoids_fs = PAM(X.values, k=2, seed=best_seed_fs)
predictions_fs = get_predictions(clusters_fs)
cm_kmeans = confusion_matrix(y, predictions_kmeans)
cm_fs = confusion_matrix(y, predictions_fs)
print(f'confusion matrix kmeans:\n{cm_kmeans}')
print(f'confusion matrix farthest first:\n{cm_fs}')

First medoid randomly chosen: 3
list_medoids: [3]
distance_matrix = [[0, 1, 2, 4, 5, 6, 6, 2, 3, 5, 4, 5, 3, 5], [1, 0, 3, 5, 6, 5, 5, 3, 4, 6, 3, 4, 4, 4], [2, 3, 0, 4, 5, 6, 4, 4, 5, 5, 6, 3, 1, 5], [4, 5, 4, 0, 3, 4, 6, 2, 5, 1, 4, 3, 5, 1], [5, 6, 5, 3, 0, 1, 3, 5, 2, 2, 5, 6, 4, 4], [6, 5, 6, 4, 1, 0, 2, 6, 3, 3, 4, 5, 5, 3], [6, 5, 4, 6, 3, 2, 0, 6, 3, 5, 4, 3, 3, 5], [2, 3, 4, 2, 5, 6, 6, 0, 3, 3, 2, 3, 5, 3], [3, 4, 5, 5, 2, 3, 3, 3, 0, 4, 3, 6, 4, 6], [5, 6, 5, 1, 2, 3, 5, 3, 4, 0, 3, 4, 4, 2], [4, 3, 6, 4, 5, 4, 4, 2, 3, 3, 0, 3, 5, 3], [5, 4, 3, 3, 6, 5, 3, 3, 6, 4, 3, 0, 4, 2], [3, 4, 1, 5, 4, 5, 3, 5, 4, 4, 5, 4, 0, 6], [5, 4, 5, 1, 4, 3, 5, 3, 6, 2, 3, 2, 6, 0]]

Medoids after farthest-first: [3, 9]
Clusters after first iteration: [[3, 0, 1, 2, 7, 11, 13], [9, 4, 5, 6, 8, 10, 12]]
intra_cluster_variances = [19, 21]

*** SWAP phase ***
best_swap = 4 > 0, performing swapping with indexes (1, 4)
medoids = [3, 9]
new medoids: [3, 4]
intra_cluster_variances = [19, 17]

best_sw

#### 7.4 Chosir un modèle obtenu par K-means
> 1. Chargement des données et preprocessing.

In [29]:
path_vote = './vote.csv'
df_vote = pd.read_csv(path_vote)
df_vote.head(3)

,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa,Class
0,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y,republican
1,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?,republican
2,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n,democrat


On remarque les valeurs sont des lettres. On va les changer en 0 ou 1. <br> 
Pour les valeurs manquantes ('?'), on les met à None.

In [30]:
mapping = {'y': 1, 'n':0, '?':None, 'republican':1, 'democrat':0}
df_vote = df_vote.transform(lambda x: x.apply(lambda c: mapping[c]))
incomplete_rows = df_vote.isnull().any(axis=1).sum()
print(f'There are {incomplete_rows} with at least 1 missing value, out of {len(df_vote)} rows')

There are 203 with at least 1 missing value, out of 435 rows


In [35]:
print(f'len(df_vote = {len(df_vote)})')

len(df_vote = 435)


La base de donnée contient de nombreuses lignes avec au moins une valeur manquante. <br>
Deux choix sont possibles: <br>
> 1. Supression des lignes qui contiennent au moins une valeur manquante. <br>
> 2. Ajout d'une valeur par interpolation, par exemple plus proches voisins. <br>

Comme près de la moitié des lignes contiennent au moins une valeur manquante, une interpolation va probablement fausser les données. <br>
L'option 1. est donc choisi: les lignes qui contiennent au moins une valeur manquante sont supprimées. 

In [36]:
df_vote.dropna(inplace=True)
columns_vote = df_vote.columns
X_vote, y_vote = df_vote[columns_vote[:-1]], df_vote[columns_vote[-1]]
print(f'columns_vote = {columns_vote}')

columns_vote = Index(['handicapped-infants', 'water-project-cost-sharing',
       'adoption-of-the-budget-resolution', 'physician-fee-freeze',
       'el-salvador-aid', 'religious-groups-in-schools',
       'anti-satellite-test-ban', 'aid-to-nicaraguan-contras', 'mx-missile',
       'immigration', 'synfuels-corporation-cutback', 'education-spending',
       'superfund-right-to-sue', 'crime', 'duty-free-exports',
       'export-administration-act-south-africa', 'Class'],
      dtype='object')


> 2 et 3. Échantillonnage de 10%

In [53]:
_, X_10, _, y_10 = train_test_split(X_vote, y_vote, test_size=0.1, random_state=1)
assert len(X_10) == len(y_10)
print(f'len(x10) = {len(X_10)}')

len(x10) = 24


> 4. Échantillonnage de 90%. <br>
Le random state est changé pour que les deux échantillonnages soient bien distincts.

In [54]:
_, X_90, _, y_90 = train_test_split(X_vote, y_vote, test_size=0.9, random_state=2)
assert len(X_90) == len(y_90)
print(f'len(x90) = {len(X_90)}')

len(x90) = 209


> 5. Application des 4 K-means <br>
(a) Application à la base de données entière

> J'ai relevé l'accuracy pour évaluer les performances du modèles.

In [46]:
kmeans_all = KMeans(n_clusters=2, random_state=0, init='random', n_init='auto').fit(X_vote)
predictions_all = kmeans_all.labels_
accuracy_all = accuracy_score(y_vote, predictions_all)

0.896551724137931

> (b) split 0.66

In [52]:
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(X_vote, y_vote, test_size=0.33, random_state=2)
kmeans_b = KMeans(n_clusters=2, random_state=0, init='random', n_init='auto').fit(X_train_b)
predictions_train_b = kmeans_b.labels_
predictions_test_b = kmeans_b.predict(X_test_b)
accuracy_b_train = accuracy_score(y_train_b, kmeans_b.labels_)
accuracy_b_test = accuracy_score(y_test_b, predictions_test_b)
print(f'accuracy_train_b = {accuracy_b_train}')
print(f'accuracy_b_test = {accuracy_b_test}')

accuracy_train_b = 0.8580645161290322
accuracy_b_test = 0.961038961038961


> (c) split

In [56]:
kmeans_c = KMeans(n_clusters=2, random_state=0, init='random', n_init='auto').fit(X_90)
predictions_test_c = kmeans_c.predict(X_10)
accuracy_c_test = accuracy_score(y_10, predictions_test_c) 

0.9166666666666666

> 6.

In [58]:
X_train_6, X_test_6, y_train_6, y_test_6 = train_test_split(X_vote, y_vote, test_size=0.1, random_state=5)
kmeans_6 = KMeans(n_clusters=2, random_state=0, init='random', n_init='auto').fit(X_train_6)
predictions_train_6 = kmeans_6.labels_
predictions_test_6 = kmeans_6.predict(X_test_6)
accuracy_6_train = accuracy_score(y_train_6, kmeans_6.labels_)
accuracy_6_test = accuracy_score(y_test_6, predictions_test_6)
print(f'accuracy_train_6 = {accuracy_6_train}')
print(f'accuracy_6_test = {accuracy_6_test}')

accuracy_train_6 = 0.8990384615384616
accuracy_6_test = 0.8333333333333334


> L'accuracy de 6. n'est pas très proche de l'accuracy de 5.c), probablement car la seed a été changée entre les deux. <br> 
Il est possible que Weka ne change pas la seed, ce qui conduit à des erreurs plus proches car les ensembles test / entrainement sont alors similaires.

> 7.

Le modèle b a la meilleure test accuracy: 96%. <br>
C'est aussi le modèle le mieux testé car il a l'ensemble de test le plus grand. <br> 
On conserve donc le modèle b.